<a href="https://colab.research.google.com/github/eTrovamala/Diplomado-Data-Science/blob/main/Proyecto_Predicci%C3%B3n_de_renuncias.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

EL objetivo de las siguientes lineas de código es el poder limpiar la base de datos, seleccionando las variables más adecuadas para poder predecir si las personas pueden o nó tender a renunciar.

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from imblearn.over_sampling import SMOTE

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
# Se comienza leyendo el DataFrame para tratar los datos.

EmpleadosAttrition = pd.read_csv('/content/gdrive/MyDrive/Diplomado Data Science/Machine Learning/Ingeniería de las características/empleadosRETO.csv')

EmpleadosAttrition.head()

,Age,BusinessTravel,Department,DistanceFromHome,Education,EducationField,EmployeeCount,EmployeeNumber,EnvironmentSatisfaction,Gender,...,PercentSalaryHike,PerformanceRating,RelationshipSatisfaction,StandardHours,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsInCurrentRole,YearsSinceLastPromotion,Attrition
0,50,Travel_Rarely,Research & Development,1 km,2,Medical,1,997,4,Male,...,22,4,3,80,32,1,2,4,1,No
1,36,Travel_Rarely,Research & Development,6 km,2,Medical,1,178,2,Male,...,20,4,4,80,7,0,3,2,0,No
2,21,Travel_Rarely,Sales,7 km,1,Marketing,1,1780,2,Male,...,13,3,2,80,1,3,3,0,1,Yes
3,52,Travel_Rarely,Research & Development,7 km,4,Life Sciences,1,1118,2,Male,...,19,3,4,80,18,4,3,6,4,No
4,33,Travel_Rarely,Research & Development,15 km,1,Medical,1,582,2,Male,...,12,3,4,80,15,2,4,6,7,Yes


In [ ]:
# Se procede a hacer eliminación de columnas que sean irrelevamtes y que no tengan correlación con la variable de estudio.
# En este caso son las columnas EmployeeCount, EmployeeNumber, Over18 y StandardHours.

EmpleadosAttrition.drop(columns=['EmployeeCount', 'EmployeeNumber', 'Over18', 'StandardHours'], inplace=True)

In [ ]:
# Como podemos observar en las características del DataFrame nos hace falta los años que llevan laborando en la empresa
# Obtendremos de la columna HiringDate el año y como los datos son string y tienen diferentes formatos de fecha, extraemos los ultimos 4 digitos
EmpleadosAttrition['Year'] = EmpleadosAttrition['HiringDate'].str[-4:].astype(int)

# El limite marcado sera de 2018, por lo que se tendra que restar 2018 al dato obtendio
EmpleadosAttrition['TearAtCompany'] = 2018 - EmpleadosAttrition['Year']

# Eliminemos las caracteristicas que ya no nos son útiles, en este caso es Year y HiringDate
EmpleadosAttrition.drop(columns=['Year', 'HiringDate'], inplace=True)

In [ ]:
# Haciendo un análisis de la columna DistanceFromHome, obsrvamos que tiene texto en string el cual no nos ayuda para el análisis numérico.
#Primero hay que convertir la columna completa a un string antes de hacer la limpieza
EmpleadosAttrition['DistanceFromHome'] = EmpleadosAttrition['DistanceFromHome'].astype(str)

# Despues se busca en cada valor de la columna cualquier número, incluyendo decimales, ignorando el texto y que solo devuelva el número encontrado
EmpleadosAttrition['DistanceFromHome'] = EmpleadosAttrition['DistanceFromHome'].str.extract(r'(\d+\.?\d*)')

# Convertimos eñ número encontrado de string a float
EmpleadosAttrition['DistanceFromHome'] = EmpleadosAttrition['DistanceFromHome'].astype(int)

# Se cambia el nombre de la columna para que aparezca las unidades desde el nombre de la columna
EmpleadosAttrition.rename(columns={'DistanceFromHome':'DistanceFromHome_km'}, inplace=True)

In [ ]:
# Analizando las caracteristicas observamos que la columna MonthlyIncome tiene valores muy grandes en comparación por lo que se escalará
# Al solo haber valores positivos se usara el método Min-Max
escalador = MinMaxScaler()
frame_escalado = escalador.fit_transform(EmpleadosAttrition[['MonthlyIncome']])
EmpleadosAttrition['MonthlyIncome'] = frame_escalado
EmpleadosAttrition.rename(columns={'MonthlyIncome':'MonthlyIncome_escalado'}, inplace=True)

In [ ]:
# Podemos observar que hay variables categóricas importantes, por lo cual se va a convertir a valores booleanos
EmpleadosAttrition['Attrition'] = EmpleadosAttrition['Attrition'].astype(str).str.strip().map({'Yes': 1, 'No': 0})
categoricas = ['BusinessTravel', 'Department', 'EducationField', 'Gender', 'JobRole', 'MaritalStatus', 'OverTime']
EmpleadosAttrition = pd.get_dummies(EmpleadosAttrition, columns=categoricas, drop_first=False)

In [ ]:
# Se debe de escoger las mejores variables, para ello se realizara un ejercicio de correlación.
# El mejor método es SelctKBest (f_classif), ya que es automático, ideal para clasificacion, facil de interpretar.
# Se debe de separar primero las variables de entrada (X) y la de salida (y)
y = EmpleadosAttrition['Attrition']  # variables de salida
X = EmpleadosAttrition.drop(columns=['Attrition'])  # Variables de entrada

# Seleccionar solo columnas numéricas (considerará las columnas con datos booleanos, float e int)
X_num = X.select_dtypes(include=['int64', 'float64', 'bool'])

# Se aplica SlectKBest con ANOVA
selector = SelectKBest(score_func=f_classif, k='all')
selector.fit(X_num, y)

# Extraer p-values y seleccionar solo las variables que son significativas se usa un alpha de 0.05
pvals = pd.Series(selector.pvalues_, index=X_num.columns)
vars_significativas = pvals[pvals < 0.05].index.tolist()

print('Variables seleccionadas automáticamente:')
print(vars_significativas)

# Creando el nuevo DataFrame filtrado ya automáticamente
EmpleadosAttritionFinal = X_num[vars_significativas].copy()
EmpleadosAttritionFinal['Attrition'] = y.values

EmpleadosAttritionFinal.head()

Variables seleccionadas automáticamente:
['Age', 'EnvironmentSatisfaction', 'JobInvolvement', 'JobLevel', 'JobSatisfaction', 'MonthlyIncome_escalado', 'TotalWorkingYears', 'YearsInCurrentRole', 'TearAtCompany', 'BusinessTravel_Non-Travel', 'EducationField_Technical Degree', 'JobRole_Healthcare Representative', 'JobRole_Laboratory Technician', 'JobRole_Research Director', 'JobRole_Sales Representative', 'MaritalStatus_Divorced', 'MaritalStatus_Single', 'OverTime_No', 'OverTime_Yes']


,Age,EnvironmentSatisfaction,JobInvolvement,JobLevel,JobSatisfaction,MonthlyIncome_escalado,TotalWorkingYears,YearsInCurrentRole,TearAtCompany,BusinessTravel_Non-Travel,EducationField_Technical Degree,JobRole_Healthcare Representative,JobRole_Laboratory Technician,JobRole_Research Director,JobRole_Sales Representative,MaritalStatus_Divorced,MaritalStatus_Single,OverTime_No,OverTime_Yes,Attrition
0,50,4,3,4,4,0.864269,32,4,5,False,False,False,False,True,False,True,False,True,False,0
1,36,2,3,2,2,0.207340,7,2,3,False,False,False,False,False,False,True,False,True,False,0
2,21,2,3,1,2,0.088062,1,0,1,False,False,False,False,False,True,False,True,True,False,1
3,52,2,3,3,2,0.497574,18,6,8,False,False,True,False,False,False,False,True,True,False,0
4,33,2,3,3,3,0.664470,15,6,7,False,False,False,False,False,False,False,False,False,True,1


In [ ]:
# Para la generacion de PCA y reducir asi la dimensionalidad se realiza el siguiente codigo
usa_pca = True # Cambiar a True si se quiere usar

if usa_pca:
  pca = PCA(n_components=0.95)  # 95% de varianza, definimos que queremos los componentes hasta obtener un 95%
  X_pca = pca.fit_transform(EmpleadosAttritionFinal.drop(columns=['Attrition']))

  print(f'\nNúmero de componentes: {X_pca.shape[1]}')
else:
  X_pca = EmpleadosAttritionFinal.drop(columns=['Attrition'])

  # El resultado nos indica que necesitamos al menos 3 columnas para explicar el 95% de la varianza.


Número de componentes: 3


In [ ]:
# Para ejecutar PCA usando solo el número de componentes
pca_final = PCA(n_components=X_pca.shape[1])
pca_data = pca_final.fit_transform(EmpleadosAttritionFinal.drop(columns=['Attrition']))

# Ahora creamos con las columnas que describa todo el sistema se crea un DF llamado EmpleasoAttritionPCA
EmpleadosAttritionPCA = pd.DataFrame(pca_data, columns=[f'PC{i+1}' for i in range(X_pca.shape[1])])
EmpleadosAttritionPCA['Attrition'] = EmpleadosAttritionFinal['Attrition']

EmpleadosAttritionPCA.head()

,PC1,PC2,PC3,Attrition
0,20.252631,-4.207105,10.720609,0
1,-6.383414,-3.691986,-0.155441,0
2,-21.507017,1.745627,3.220579,1
3,13.826913,-5.844031,-2.225039,0
4,-1.428875,4.147262,3.921176,1


In [ ]:
# Ahora si queremos juntar las columnas de PCA con el Frame original se concatenan
EmpleadosAttritionFinal = pd.concat([EmpleadosAttritionPCA, EmpleadosAttritionFinal], axis=1)

EmpleadosAttritionFinal.head()

,PC1,PC2,PC3,Attrition,Age,EnvironmentSatisfaction,JobInvolvement,JobLevel,JobSatisfaction,MonthlyIncome_escalado,...,EducationField_Technical Degree,JobRole_Healthcare Representative,JobRole_Laboratory Technician,JobRole_Research Director,JobRole_Sales Representative,MaritalStatus_Divorced,MaritalStatus_Single,OverTime_No,OverTime_Yes,Attrition
0,20.252631,-4.207105,10.720609,0,50,4,3,4,4,0.864269,...,False,False,False,True,False,True,False,True,False,0
1,-6.383414,-3.691986,-0.155441,0,36,2,3,2,2,0.207340,...,False,False,False,False,False,True,False,True,False,0
2,-21.507017,1.745627,3.220579,1,21,2,3,1,2,0.088062,...,False,False,False,False,True,False,True,True,False,1
3,13.826913,-5.844031,-2.225039,0,52,2,3,3,2,0.497574,...,False,True,False,False,False,False,True,True,False,0
4,-1.428875,4.147262,3.921176,1,33,2,3,3,3,0.664470,...,False,False,False,False,False,False,False,False,True,1


In [ ]:
# Para entrenar un modelo que nos ayude a predecir el comportamiento

X_train, X_test, y_train, y_test = train_test_split(X_pca, y, test_size=0.20, random_state=42, stratify=y)

modelo = LogisticRegression(max_iter=500)
modelo.fit(X_train, y_train)

y_pred = modelo.predict(X_test)

In [ ]:
# Para evaluar el modelo que se generó

print("\n==== RESULTADOS ====\n")
print("Accuracy:", accuracy_score(y_test, y_pred)) #Calcula la precisión general, es decir, el porcentaje de predicciones correctas sobre todas las predicciones
print("\nClassification report:\n", classification_report(y_test, y_pred))
print("\nConfusion matrix:\n", confusion_matrix(y_test, y_pred))


==== RESULTADOS ====

Accuracy: 0.85

Classification report:
               precision    recall  f1-score   support

           0       0.85      1.00      0.92        67
           1       1.00      0.08      0.14        13

    accuracy                           0.85        80
   macro avg       0.92      0.54      0.53        80
weighted avg       0.87      0.85      0.79        80


Confusion matrix:
 [[67  0]
 [12  1]]


In [ ]:
# Para hacer mas preciso el modelo de predicción se empleara las siguientes modificaciones
from xgboost import XGBClassifier

X_train, X_test, y_train, y_test = train_test_split(X_pca, y, test_size=0.20, random_state=42, stratify=y)

# Se escalan los datos
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Se aplica SMOTE (balancear la clase 1 de la variable de respuesta)
sm = SMOTE(random_state=42)
X_train_bal, y_train_bal = sm.fit_resample(X_train_scaled, y_train)

modelo = XGBClassifier(scale_pos_weight=5)
modelo.fit(X_train_bal, y_train_bal)

y_pred = modelo.predict(X_test_scaled)

# Para evaluar el modelo que se generó

print("\n==== RESULTADOS ====\n")
print("Accuracy:", accuracy_score(y_test, y_pred)) #Calcula la precisión general, es decir, el porcentaje de predicciones correctas sobre todas las predicciones
print("\nClassification report:\n", classification_report(y_test, y_pred))
print("\nConfusion matrix:\n", confusion_matrix(y_test, y_pred))


==== RESULTADOS ====

Accuracy: 0.6

Classification report:
               precision    recall  f1-score   support

           0       0.83      0.66      0.73        67
           1       0.15      0.31      0.20        13

    accuracy                           0.60        80
   macro avg       0.49      0.48      0.47        80
weighted avg       0.72      0.60      0.65        80


Confusion matrix:
 [[44 23]
 [ 9  4]]


In [ ]:
# Para hacer mas preciso el modelo de predicción se empleara las siguientes modificaciones
from xgboost import XGBClassifier

X_train, X_test, y_train, y_test = train_test_split(X_pca, y, test_size=0.20, random_state=42, stratify=y)

# Se escalan los datos
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Se aplica SMOTE (balancear la clase 1 de la variable de respuesta)
sm = SMOTE(random_state=42)
X_train_bal, y_train_bal = sm.fit_resample(X_train_scaled, y_train)

# Modelo XGBOOST optimizado
xgb = XGBClassifier(objective='binary:logistic',
                    eval_metric='logloss',
                    learrning_rate=0.05,
                    n_estimators=300,
                    max_depth=4,
                    subsample=0.8,
                    solsample_bytree=0.8,
                    gama=1.0,
                    random_state=42)

xgb.fit(X_train_bal, y_train_bal)

# Ajustar Threshold
y_prob = xgb.predict_proba(X_test_scaled)[:,1]
threshold = 0.35 # baja para mejorar recall
y_pred = (y_prob >= threshold).astype(int)

# Para evaluar el modelo que se generó

print("\n==== RESULTADOS ====\n")
print("Accuracy:", accuracy_score(y_test, y_pred)) #Calcula la precisión general, es decir, el porcentaje de predicciones correctas sobre todas las predicciones
print("\nClassification report:\n", classification_report(y_test, y_pred))
print("\nConfusion matrix:\n", confusion_matrix(y_test, y_pred))

/usr/local/lib/python3.12/dist-packages/xgboost/training.py:199: UserWarning: [23:31:54] WARNING: /workspace/src/learner.cc:790: 
Parameters: { "gama", "learrning_rate", "solsample_bytree" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)



==== RESULTADOS ====

Accuracy: 0.6375

Classification report:
               precision    recall  f1-score   support

           0       0.83      0.72      0.77        67
           1       0.14      0.23      0.17        13

    accuracy                           0.64        80
   macro avg       0.48      0.47      0.47        80
weighted avg       0.72      0.64      0.67        80


Confusion matrix:
 [[48 19]
 [10  3]]
